In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('preprocessed.tsv',delimiter='\t')

In [4]:
df.head()

,Unnamed: 0,Unnamed: 0.1,rating,reviews,synopsis,title,user_ratings
0,0,0,9.217045,['Tied for the best movie I have ever seen Why...,1947 Andy Dufresne Tim Robbins banker Maine co...,The Shawshank Redemption,"['10/10', '10/10', '10/10', '10/10', '10/10', ..."
1,1,1,1.997582,"['The fact that aspiring actors, directors, an...",movie open spoof Armageddon 10 000 B C year 10...,Disaster Movie,"['1/10', '1/10', '1/10', '1/10', '1/10', '1/10..."
2,2,2,8.500000,['Unlike anything ever done in the history of ...,SPOILER Thanos Children Proxima Midnight Ebony...,Avengers: Infinity War,"['10/10', '10/10', '10/10', '9/10', '10/10', '..."


In [5]:
len(df)

3

In [6]:
from ast import literal_eval

In [7]:
df['reviews'] = df['reviews'].apply(literal_eval)    

In [77]:
max_len_synopsis=0
max_len_reviews=0 
no_of_reviews=0
tot_no_of_reviews=0
fin_text=''
for index,row in df.iterrows():
    max_len_synopsis = max(max_len_synopsis, len(row['synopsis']))
    text=''.join(row['synopsis'])
    fin_text+=' '+text
for i in range(0, len(df)):
    temp = df.iloc[i]['reviews']
    no_of_reviews=0;
    for text in temp:
        max_len_reviews = max(max_len_reviews, len(text))
        no_of_reviews=no_of_reviews+1
        fin_text+=' '+text
    tot_no_of_reviews += no_of_reviews
print(tot_no_of_reviews)
chars = tuple(set(fin_text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
fortyforty = (int)((max_len_reviews-36)/5)+1

#print(fortyforty)
max_len_reviews = 36+5*fortyforty
#print(max_len_reviews)

142


In [78]:
def one_hot_encoder(text):
    encoded = np.array([char2int[ch] for ch in text])
    ans = np.zeros((max_len_synopsis, len(int2char)), dtype = np.float32)
    cnt=0;
    for x in encoded:
        ans[cnt][x] = 1.0
        cnt=cnt+1
    return ans

In [79]:
print(int2char)

{0: 'j', 1: 'w', 2: '3', 3: ' ', 4: '=', 5: ',', 6: '%', 7: 'B', 8: '2', 9: 'ê', 10: 'A', 11: '&', 12: 'S', 13: 'E', 14: 'W', 15: 'G', 16: ':', 17: 'Q', 18: '#', 19: 'i', 20: '6', 21: 'K', 22: 'Y', 23: '\n', 24: '9', 25: 'R', 26: 'V', 27: 'p', 28: '\x96', 29: 'C', 30: 'm', 31: 'c', 32: 't', 33: 'z', 34: 'I', 35: '~', 36: 'a', 37: '1', 38: 'd', 39: '[', 40: '+', 41: 's', 42: '8', 43: '(', 44: 'q', 45: 'v', 46: '.', 47: 'n', 48: '4', 49: '$', 50: 'Z', 51: 'F', 52: 'é', 53: 'J', 54: "'", 55: 'D', 56: '"', 57: 'l', 58: 'T', 59: 'U', 60: 'b', 61: 'o', 62: '7', 63: ')', 64: 'O', 65: 'X', 66: ';', 67: 'f', 68: 'e', 69: 'y', 70: '0', 71: 'k', 72: 'g', 73: '-', 74: '!', 75: '5', 76: ']', 77: 'M', 78: '*', 79: 'h', 80: '`', 81: 'L', 82: 'N', 83: 'P', 84: 'r', 85: 'u', 86: 'x', 87: 'H', 88: '/', 89: '?'}


In [80]:
one_hot = one_hot_encoder("jw3e ???? is smart")
print(one_hot)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [81]:
Y = np.zeros((1, fortyforty, len(int2char)), dtype=np.float32)

In [82]:
def char_encoding2arr(c):
    encoded = np.zeros(len(int2char), dtype = np.float32)
    encoded[char2int[c]]=1.0
    return encoded

In [83]:
cnt=0
for i in range(1):
    for j in range(1):
        for k in range(40, len(df.iloc[i]['reviews'][j]), 5):
#             print(k)
            l = (int)((k-40)/5)
            #print(l)
            Y[cnt][l] = char_encoding2arr(df.iloc[i]['reviews'][j][k])
        cnt+=1
print(df.iloc[0]['reviews'][0][0])
print(Y[0][0])

T
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [84]:
print(df.iloc[0]['reviews'][0][40])
print(df.iloc[0]['reviews'][0][45])
print(Y[0][1])

 
d
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [85]:
#print(df.iloc[0]['reviews'][0])
# len_of_review = len(df.iloc[0]['reviews'][0])
# fortyforty = (int)((len_of_review -38)/3)+1
# len_of_review  = 38+3*fortyforty
X = np.zeros((1,int(fortyforty), 40+max_len_synopsis, len(int2char)), dtype = np.float32)
X.shape

(1, 1499, 11841, 90)